In [2]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

import joblib
from utils import make_dataset, display_results

warnings.filterwarnings('ignore')

TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [3]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [4]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_train_pred = clf.predict_proba(X_train)
train['recid_score'] = (y_train_pred[:,1])
train['recid_prediction'] = (y_train_pred[:,1] >0.5).astype(int)

y_test_pred = clf.predict_proba(X_test)
test['recid_score'] = (y_test_pred[:,1])
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

In [5]:
cols = FEATURE_NAMES+[TARGET_COLS, 'recid_prediction','recid_score']

ground_truth_train = make_dataset(train[cols], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_train = make_dataset(train[cols], 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

ground_truth_test = make_dataset(test[cols], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test[cols], 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

# Equal Odds

## Method

## Pros and Cons

## Materials

In [10]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

calibrator = EqOddsPostprocessing(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [11]:
joblib.dump((clf_metric,roc_auc), '../results/1.1-equal_odds.pkl')
display_results('../results/1.1-equal_odds.pkl')

,metric_names,scores
0,roc_auc_score,0.501225
1,true_positive_rate_difference,-0.001722
2,false_positive_rate_difference,0.000000
3,false_omission_rate_difference,-0.037656
4,false_discovery_rate_difference,0.000000
5,error_rate_difference,-0.036975
6,false_positive_rate_ratio,NaN
7,false_negative_rate_ratio,1.001728
8,false_omission_rate_ratio,0.936201
9,false_discovery_rate_ratio,NaN


# Calibrated Equal Odds

## Method

## Pros and Cons

## Materials

In [8]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

calibrator = CalibratedEqOddsPostprocessing(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [9]:
joblib.dump((clf_metric,roc_auc), '../results/1.1-calibrated_equal_odds.pkl')
display_results('../results/1.1-calibrated_equal_odds.pkl')

,metric_names,scores
0,roc_auc_score,0.648129
1,true_positive_rate_difference,-0.134699
2,false_positive_rate_difference,-0.319859
3,false_omission_rate_difference,-0.030586
4,false_discovery_rate_difference,-0.057899
5,error_rate_difference,-0.045089
6,false_positive_rate_ratio,0.559052
7,false_negative_rate_ratio,2.212291
8,false_omission_rate_ratio,0.917122
9,false_discovery_rate_ratio,0.839538


# Rejection Option

## Method

## Pros and Cons

## Materials

In [6]:
from aif360.algorithms.postprocessing import RejectOptionClassification

calibrator = RejectOptionClassification(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [7]:
joblib.dump((clf_metric,roc_auc), '../results/1.1-rejection_option.pkl')
display_results('../results/1.1-rejection_option.pkl')

,metric_names,scores
0,roc_auc_score,0.653778
1,true_positive_rate_difference,-0.102441
2,false_positive_rate_difference,-0.247320
3,false_omission_rate_difference,-0.035370
4,false_discovery_rate_difference,-0.042279
5,error_rate_difference,-0.034496
6,false_positive_rate_ratio,0.621169
7,false_negative_rate_ratio,1.714525
8,false_omission_rate_ratio,0.905385
9,false_discovery_rate_ratio,0.877524
